## PLEIADES Tutorial - SAMMY Exercise EX012: Treating Multiple Nuclides in a Sample

### Overview

This notebook provides a comprehensive guide to SAMMY Exercise EX012 from the PLEIADES examples. The exercise demonstrates how to handle samples containing multiple nuclides, a scenario frequently encountered in real-world experiments. By leveraging PLEIADES, users will learn to configure, execute, and analyze SAMMY runs for such complex cases.

### Objectives

- Understand how to use PLEIADES to manage SAMMY runs for samples with multiple nuclides.
- Learn to define spin groups, resonance parameters, and nuclide abundances.
- Explore the impact of individual isotopes or spin groups on the overall sample.
- Refine resonance parameters and nuclide abundances to improve the fit to experimental data.

### Key Details

1. **Using PLEIADES**:
    - This notebook demonstrates the step-by-step process of setting up and executing SAMMY runs using PLEIADES.
    - It includes configuring the environment, linking necessary files, running SAMMY, and analyzing the results.

2. **Handling Multiple Nuclides**:
    - The exercise focuses on treating samples with multiple isotopes or contaminants.
    - Users will learn to exclude specific spin groups and analyze their contributions to the overall sample.

3. **Practical Insights**:
    - Gain insights into how resonance parameters and spin groups influence experimental results.
    - Experiment with different configurations to understand the behavior of the sample.

### Instructions

Follow the steps in this notebook to configure the SAMMY environment, link the required files, and execute the SAMMY runs. Analyze the results to refine nuclide abundances and resonance parameters. Experiment with excluding specific spin groups to observe their individual contributions and deepen your understanding of treating multiple nuclides in a sample.

### PRE-SETUP: Initializing Directories and Files for the Example

To begin, we need to set up a temporary directory and link all the required SAMMY files to it. This ensures that the necessary files are organized and accessible for running the SAMMY exercise.

In this step, we will create symbolic links for the required files from the `ex012` example into the designated `working_dir`. These files are essential for configuring and executing the SAMMY run.

The directories are configured as follows:
- **Working Directory**: `/tmp/sammy_run`
- **Output Directory**: `/tmp/sammy_run/sammy_output`

The following files should be available in the `working_dir`:
- `ex012a.inp`: Input file defining spin groups for different isotopes.
- `ex012a.par`: Parameter file containing resonance data grouped by nuclide.
- `ex012a.dat`: Data file with "real data" for transmission through natural silicon.

Ensure that these files are correctly linked to the `working_dir` before proceeding with the analysis.

In [ ]:
from pathlib import Path
import tempfile

# Get current working directory (notebook location)
current_dir = Path.cwd()

# Create a dedicated sammy_workspaces folder for all runs
sammy_base_dir = current_dir / "workspaces"
sammy_base_dir.mkdir(exist_ok=True)

# Create a timestamped working directory within our persistent folder
base_dir_name = f"Si_Transmission"
working_dir = sammy_base_dir / base_dir_name
output_dir = working_dir / "results"

# Create the directories if they do not exist
working_dir.mkdir(parents=True, exist_ok=True)
output_dir.mkdir(parents=True, exist_ok=True)

# Copy the ex012a.inp, ex012a.par, and ex012a.dat files from "../samexm/ex012/" to the working directory
# Define the source directory and the files to copy
source_dir = Path("../samexm/ex012/").resolve()
files_to_link = ["ex012a.inp", "ex012a.par", "ex012a.dat"]

# Create symbolic links for each file in the working directory
for file_name in files_to_link:
    source_file = source_dir / file_name
    destination_file = working_dir / file_name
    
    # Check if symlink exists already and if so, remove it
    if destination_file.exists():
        if destination_file.is_symlink():
            destination_file.unlink()  # Remove the existing symbolic link
    destination_file.symlink_to(source_file)

# Verify the files are in the working directory
print("Files in working directory:", list(working_dir.iterdir()))

### Loading Relevant Modules

This section focuses on importing the necessary modules from PLEIADES and other libraries to configure and execute the SAMMY exercise. These modules are essential for managing SAMMY input/output files, setting up the working environment, and running SAMMY locally.

In [ ]:
import subprocess

from pleiades.sammy.config import LocalSammyConfig              # Needed for configuring the SAMMY run directory 
from pleiades.sammy.backends.local import LocalSammyRunner      # Needed for running SAMMY locally
from pleiades.sammy.interface import SammyFiles                 # Needed for managing SAMMY input and output files 

### Configuring the SAMMY environment with PLEIADES

In this step, we set up the SAMMY environment by specifying the executable path, working directory, and output directory. The `LocalSammyConfig` class is used to create a configuration object that holds these details. This configuration will be used to manage the SAMMY runs effectively.

In [ ]:
# Grab the SAMMY executable 
sammy_executable = subprocess.run(["which", "sammy"], capture_output=True, text=True).stdout.strip()

# Create SAMMY configuration class
config = LocalSammyConfig(
    sammy_executable=sammy_executable,
    working_dir=working_dir,
    output_dir=output_dir
)

print(f"Using SAMMY executable: {config.sammy_executable}")
print(f"Using working directory: {config.working_dir}")
print(f"Using output directory: {config.output_dir}")

# Double check the success of configuration through PLEIADES validation
if config.validate():
    print("Configuration validated successfully.")
else:
    print("Configuration validation failed.")


### Configuring SAMMY Files

Now set the needed SAMMY files using the `SammyFiles` class. This class allows us to define the paths for the input, parameter, and data files required for the SAMMY run. These files are located in the `working_dir` and were linked earlier in the setup process.

The following files are configured:
- **Input File**: `ex012a.inp` - Defines spin groups for different isotopes.
- **Parameter File**: `ex012a.par` - Contains resonance data grouped by nuclide.
- **Data File**: `ex012a.dat` - Provides "real data" for transmission through natural silicon.

These files are essential for running the SAMMY exercise and analyzing the results.

In [ ]:
# Define input files
files = SammyFiles(
    input_file=working_dir / "ex012a.inp",
    parameter_file=working_dir / "ex012a.par",
    data_file=working_dir / "ex012a.dat"
)

print("SAMMY FILES:")
print(f"Input file: {files.input_file}")
print(f"Parameter file: {files.parameter_file}")
print(f"Data file: {files.data_file}")

### Executing SAMMY with Configured Files

In this step, we will execute SAMMY using the `SammyFiles` configuration within the `working_dir`. The `LocalSammyRunner` class will be utilized to run SAMMY locally, leveraging the previously defined `config` and `files` objects.

The execution process involves:
- Using the `LocalSammyRunner` to manage the SAMMY run.
- Ensuring the input, parameter, and data files are correctly linked and accessible in the `working_dir`.
- Running SAMMY with the specified configuration and files.

This step is crucial for generating results based on the provided input and parameter files, which will be analyzed in subsequent steps.

In [ ]:
try:
    # Create and use runner
    runner = LocalSammyRunner(config)

    # Prepare environment
    print("Preparing environment...")
    runner.prepare_environment(files)

    # Execute SAMMY
    print("Executing SAMMY...")
    result = runner.execute_sammy(files)

    # Process results
    if result.success:
        print(f"SAMMY execution successful (runtime: {result.runtime_seconds:.2f}s)")
        runner.collect_outputs(result)

    else:
        print("SAMMY execution failed:")
        print(result.error_message)
        print("\nConsole output:")
        print(result.console_output)
    

except Exception as e:
    print(f"Error running SAMMY: {str(e)}")

finally:
    # Cleanup
    runner.cleanup()

### Load the theoretical results of the fit from the `SAMMY.LST` file. 

The results of the SAMMY fit will always be written out to a `SAMMY.LST` file. This was collected and moved to the `output_dir` when `collect_outputs()` was called.

Here we load the data from the `SAMMY.LST` file into a pandas dataframe. We can print the dataframe's head to check if the data has been loaded correctly and inspect the first few rows of the results.

In [ ]:
import pleiades.sammy.data.options as options

# grab the SAMMY.LST file from the output directory
lst_file = output_dir / "SAMMY.LST"

lstData = options.LstData(data_file=str(lst_file), data_type=options.DataTypeOptions.TRANSMISSION)
lstData.data.head()

### Plot Results

In this section, we visualize the results of the SAMMY fit by plotting the experimental and theoretical cross sections as a function of energy. Additionally, we compute and plot the residuals (difference between experimental and theoretical cross sections) to assess the quality of the fit.

The main plot displays:
- Experimental cross section (data points).
- Final theoretical cross section (continuous line).

The residuals plot shows:
- The difference between experimental and theoretical cross sections.
- A horizontal line at zero to indicate perfect agreement.

These visualizations help evaluate the accuracy of the SAMMY fit and identify any discrepancies between the experimental data and the theoretical model.

In [ ]:
import matplotlib.pyplot as plt

# Extract columns from lstData
energy = lstData.energy
exp_cs = lstData.experimental_cross_section
theo_cs = lstData.theoretical_cross_section

# Compute residuals
residual = exp_cs - theo_cs

# Create the figure with two subplots: main plot and residuals
fig, (ax1, ax2) = plt.subplots(
    2, 1, sharex=True, figsize=(8, 6), gridspec_kw={"height_ratios": [3, 1]}
)

# Main plot: Experimental and Theoretical Cross Section
ax1.plot(energy, exp_cs, 'o', label='Experimental cross section',markersize=4)
ax1.plot(energy, theo_cs, '-', label='Final theoretical cross section')
ax1.set_ylabel('Cross Section (barns)')
ax1.set_title('Experimental vs Theoretical Cross Section')
ax1.legend()
ax1.grid(True)

# Residuals plot
ax2.plot(energy, residual, 'o-', color='gray', label='Residual (Exp - Theory)')
ax2.axhline(0, color='black', linestyle='--', linewidth=1)
ax2.set_xlabel('Energy (eV)')
ax2.set_ylabel('Residual')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()